In [19]:
import requests
import os
import logging
import gdown
import random

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
from torchvision.datasets import VOCSegmentation
import torchmetrics
import torchvision
import albumentations as A

import re
import string
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import cv2
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as T
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from scipy.io import loadmat
from sklearn.manifold import TSNE
from torchmetrics.classification import MulticlassF1Score, JaccardIndex, MulticlassPrecision, MulticlassRecall, MulticlassAveragePrecision
import pandas as pd
from torchinfo import torchinfo

from transformers import ConvNextV2Model, BertModel, BertTokenizer, ViTModel, ViTConfig
from transformers import AutoTokenizer, AutoModel, RobertaTokenizer, CLIPModel, CLIPTokenizer, CLIPProcessor
from transformers import DeiTConfig, DeiTFeatureExtractor, DeiTImageProcessor, DeiTModel
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import timm
from transformers import AlignModel, AlignProcessor, AlignConfig, AlignVisionConfig, AlignTextConfig

from torchmetrics.functional import pairwise_cosine_similarity

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
logging.basicConfig(level=logging.ERROR)

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")




There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3090


In [3]:
ROOT_DIR = '../Datasets/ocular-disease-recognition-odir5k/'

## ALIGN

In [20]:
BATCH_SIZE = 16

In [5]:
CSV_PATH = ROOT_DIR + 'dataset_single_eye.csv'
TEST_CSV = ROOT_DIR + 'TESTING_dataset_single_eye.csv'
IMG_PATH = ROOT_DIR + 'preprocessed_images/'

In [6]:
torchvision.io.read_image(IMG_PATH + '0_left.jpg').shape

torch.Size([3, 512, 512])

In [7]:
train_val_df = pd.read_csv(CSV_PATH)
test_df = pd.read_csv(TEST_CSV)

In [8]:
def preprocess_text(df:pd.DataFrame):
    df['Keywords'] = df['Keywords'].str.lower()
    df['Keywords'] = df['Keywords'].apply(lambda x: " ".join(x.split()))
    df['Keywords'] = df['Keywords'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
    return df
train_val_df = preprocess_text(train_val_df)
test_df = preprocess_text(test_df)

In [9]:
np.max(train_val_df['Keywords'].apply(lambda x: len(x.split())))

10

In [10]:
train_df, val_df = train_test_split(train_val_df, test_size = 0.15, random_state= 123456)
len(train_df), len(val_df)

(4877, 861)

In [11]:
IMG_SIZE = (224, 224)

rescale_transform = torchvision.transforms.Compose([
    torchvision.transforms.CenterCrop(IMG_SIZE),
    #torchvision.transforms.Resize(IMG_SIZE, antialias = False, interpolation = torchvision.transforms.InterpolationMode.NEAREST),
    # torchvision.transforms.Normalize(
    #     timm.data.constants.IMAGENET_DEFAULT_MEAN,
    #     timm.data.constants.IMAGENET_DEFAULT_STD
    # )
])

augmentation = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(p = 0.5),
    torchvision.transforms.RandomVerticalFlip(p= 0.5),
    #torchvision.transforms.RandomRotation(90)
])

In [12]:
processor = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

Using cache found in C:\Users\krish/.cache\torch\hub\huggingface_pytorch-transformers_main


In [13]:
class ODIRDatasetMM(Dataset) :
    def __init__(self, df, IMG_FOLDER, tokenizer = processor, feature_extractor = rescale_transform, augmentation = None) :
        '''
        id : list of samples ids as string
        '''
        self.text = [tokenizer(text = x, padding = 'max_length', max_length = 45, truncation = True, return_tensors = 'pt') for x in df['Keywords']]
        self.eye = df['eye']
        self.labels = torch.tensor(df[['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']].to_numpy()).float()
        self.img_dir = [IMG_PATH + x for x in df['Image']]

        self.augmentation = augmentation

        self.images = [feature_extractor(torchvision.io.read_image(x).float()/255.0) for x in self.img_dir]
        #self.transform = transform
        
    def __len__(self):
        return len(self.images)
        
    def __getitem__(self, idx):
        batch_imgs = self.images[idx]
        if(self.augmentation is not None):
            batch_imgs = self.augmentation(batch_imgs)
        return batch_imgs, self.text[idx], self.labels[idx]

In [14]:
train_dataset = ODIRDatasetMM(train_df, IMG_PATH, augmentation = augmentation)
val_dataset   = ODIRDatasetMM(val_df, IMG_PATH)
test_dataset  = ODIRDatasetMM(test_df, IMG_PATH)

In [21]:
train_dataloader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size= BATCH_SIZE)

In [16]:
# contrastive learning on training data finetuning

In [21]:
torch.cuda.empty_cache()
o = m(pixel_values = train_image, input_ids = input_id, attention_mask = mask, return_loss = True)

In [25]:
input_id, mask

(tensor([[ 101, 3671, 4636,  ...,    0,    0,    0],
         [ 101, 6097, 7934,  ...,    0,    0,    0],
         [ 101, 3671, 4636,  ...,    0,    0,    0],
         ...,
         [ 101, 3671, 4636,  ...,    0,    0,    0],
         [ 101, 3671, 4636,  ...,    0,    0,    0],
         [ 101, 3671, 4636,  ...,    0,    0,    0]], device='cuda:0'),
 tensor([[[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         ...,
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]]], device='cuda:0'))

In [38]:
o['vision_model_output'].pooler_output

tensor([[ 0.3625, -0.2191, -0.3194,  ...,  0.2252, -0.3930,  0.3590],
        [ 0.2799, -0.1853, -0.2460,  ...,  0.2290, -0.3011,  0.4705],
        [-0.7310,  0.0430,  0.8231,  ..., -0.7190,  0.2098, -1.7877],
        ...,
        [ 0.2666, -0.1514, -0.3365,  ...,  0.3502, -0.3491,  0.4727],
        [ 0.3180, -0.2150, -0.3059,  ...,  0.2217, -0.4454,  0.4607],
        [ 0.2302, -0.2260, -0.2799,  ...,  0.2679, -0.3489,  0.4726]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [40]:
o['image_embeds']

tensor([[ 0.0502, -0.0304, -0.0443,  ...,  0.0312, -0.0545,  0.0498],
        [ 0.0349, -0.0231, -0.0307,  ...,  0.0286, -0.0376,  0.0587],
        [-0.0283,  0.0017,  0.0318,  ..., -0.0278,  0.0081, -0.0691],
        ...,
        [ 0.0370, -0.0210, -0.0467,  ...,  0.0486, -0.0484,  0.0656],
        [ 0.0398, -0.0269, -0.0383,  ...,  0.0278, -0.0558,  0.0577],
        [ 0.0290, -0.0285, -0.0353,  ...,  0.0338, -0.0440,  0.0596]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [22]:
o.keys()

odict_keys(['loss', 'logits_per_image', 'logits_per_text', 'text_embeds', 'image_embeds', 'text_model_output', 'vision_model_output'])

In [23]:
model= AlignModel(AlignConfig()).to(device)

In [24]:
torch.cuda.empty_cache()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

EPOCHS = 15

for epoch_num in range(EPOCHS):

  total_acc_train = 0
  total_loss_train = 0

  for train_image, train_text, train_label in tqdm(train_dataloader):
        optimizer.zero_grad()

        train_label = train_label.to(device)
        train_image = train_image.to(device)
        mask = train_text['attention_mask'].to(device)
        input_id = train_text['input_ids'].squeeze(1).to(device)
        output = model.forward(pixel_values = train_image, input_ids = input_id, attention_mask = mask, return_loss = True)

        batch_loss = output['loss']
        batch_loss.backward()
        optimizer.step()
        total_loss_train += batch_loss.item()
      
  total_loss_val = 0

  with torch.no_grad():
      #Validation
      for val_image, val_text, val_label in val_dataloader:

          val_label = val_label.to(device)
          val_image = val_image.to(device)
          mask = val_text['attention_mask'].to(device)
          input_id = val_text['input_ids'].squeeze(1).to(device)

          output = model.forward(pixel_values = val_image, input_ids = input_id, attention_mask = mask, return_loss = True)
          batch_loss = output['loss']
          total_loss_val += batch_loss.item()
              
      
  avg_train_loss = total_loss_train/len(train_df)

  avg_val_loss = total_loss_val/len(val_df)


  print("Epoch [{}/{}], Train Loss: {:.4f}".format(epoch_num+1, EPOCHS, avg_train_loss*BATCH_SIZE))
  print("Epoch [{}/{}], Val Loss: {:.4f}".format(epoch_num+1, EPOCHS, avg_val_loss*BATCH_SIZE))

  torch.save(model.state_dict(), './' + 'checkpoint' + '.pt' )

torch.save(model.state_dict(), './' + 'finetuned' + '.pt' )


100%|██████████| 305/305 [02:18<00:00,  2.20it/s]


Epoch [1/15], Train Loss: 2.7655
Epoch [1/15], Val Loss: 2.7661


100%|██████████| 305/305 [02:14<00:00,  2.27it/s]


Epoch [2/15], Train Loss: 2.7577
Epoch [2/15], Val Loss: 2.7615


  4%|▍         | 13/305 [00:05<02:11,  2.22it/s]


KeyboardInterrupt: 

In [ ]:
weights = torch.tensor([.7,1.2,1.5,1.5,1.5,1.7, 1.5, 1.2]).to(device)
criterion_image = nn.BCELoss(weights)
AVERAGING = 'micro'
acc_train_img = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
acc_train_text = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
acc_val_img   = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)
acc_val_text   = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)

In [ ]:
with open("finetuned.pt", 'rb') as f:
    model.load_state_dict(torch.load(f))

## Evaluation on test set

In [ ]:
criterion = nn.BCELoss(torch.tensor([0.5, 1, 5, 5, 5, 6, 5, 1]).float().to(device))

test_loss = 0
test_acc  = 0

AVERAGING = 'weighted'
PREC = torchmetrics.classification.MultilabelPrecision(8, average = AVERAGING).to(device)#, validate_args = False)
ACC = torchmetrics.classification.MultilabelAccuracy(8, average = AVERAGING).to(device)#, validate_args = False)
REC = torchmetrics.classification.MultilabelRecall(8, average = AVERAGING).to(device)#, validate_args = False)
F1_SCORE = torchmetrics.classification.MultilabelF1Score(8, average = AVERAGING).to(device)#, validate_args = False)
F_BETA_SCORE = torchmetrics.classification.MultilabelFBetaScore(beta = 0.8, num_classes = 8, num_labels = 8, average = AVERAGING).to(device)#, validate_args = False)
KAPPA = torchmetrics.classification.MulticlassCohenKappa(8).to(device)#, validate_args = False)
AUC = torchmetrics.classification.MultilabelAUROC(8, average = AVERAGING).to(device)#, validate_args = False)

for train_image, train_text, train_label in tqdm(test_dataloader): 
    with torch.no_grad():
        train_label = train_label.to(device)
        train_image = train_image.to(device)
        mask = train_text['attention_mask'].to(device)
        input_id = train_text['input_ids'].squeeze(1).to(device)
        
        # logits_per_image, logits_per_text
        #out_img, out_txt = model.forward(train_image, input_id, mask, contrastive = True)
        predictions = model.forward(train_image, contrastive = False)



        train_label = train_label.long()
        PREC(predictions, train_label)
        ACC(predictions, train_label)
        REC(predictions, train_label)
        F1_SCORE(predictions, train_label)
        F_BETA_SCORE(predictions, train_label)
        KAPPA(predictions, train_label)
        AUC(predictions, train_label)


add_prec = PREC.compute()
add_acc = ACC.compute()
add_rec = REC.compute()
add_f1 = F1_SCORE.compute()
add_fbeta = F_BETA_SCORE.compute()
add_kappa = KAPPA.compute()
add_auc = AUC.compute()

avg_test_loss = test_loss/len(test_df)*BATCH_SIZE
avg_test_acc  = test_acc /len(test_df)

print("Acc: {:3f}\nPrec: {:3f}\nRecall: {:.3f}\nF1-score: {:.3f}\nF-Beta-score: {:.3f}\nKappa: {:.3f}\nAUC: {:.3f}".format(add_acc, add_prec,add_rec, add_f1, add_fbeta, add_kappa, add_auc))
torch.cuda.empty_cache()

  0%|          | 0/13 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\torchnew\Lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
100%|██████████| 13/13 [00:02<00:00,  4.74it/s]


Acc: 0.794288
Prec: 0.626230
Recall: 0.635
F1-score: 0.627
F-Beta-score: 0.626
Kappa: 0.000
AUC: 0.816
